<a href="https://colab.research.google.com/github/Saranyanv/data-go/blob/master/Reducing%20Commercial%20Aviation%20Fatalities/EDA_PySpark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#from pyspark import SparkContext, SparkConf
from pyspark.rdd import RDD
from pyspark.sql import SparkSession
import numpy as np

In [0]:
spark = SparkSession.builder \
.master("local") \
.appName("mySparkSession") \
.config("k1", "vtest") \
.getOrCreate()

# DataFrame Creation

In [0]:
readings = spark.read.load("gs://saranv_bucket/Reducing Commercial Aviation Disaster/train.csv",
                     format="csv", sep=",", inferSchema="true", header="true")

In [0]:
print type(readings), readings.count()

<class 'pyspark.sql.dataframe.DataFrame'> 4867421


In [0]:
readings.take(5)

[Row(crew=1, experiment=u'CA', time=0.01171875, seat=1, eeg_fp1=-5.28545, eeg_f7=26.775801, eeg_f8=-9.52731, eeg_t4=-12.7932, eeg_t6=16.7178, eeg_t5=33.737499, eeg_t3=23.712299, eeg_fp2=-6.6958699999999975, eeg_o1=29.2321, eeg_p3=24.842899, eeg_pz=3.92134, eeg_f3=18.447001, eeg_fz=1.07547, eeg_f4=3.09029, eeg_c4=37.368999, eeg_p4=17.437599, eeg_poz=19.2019, eeg_c3=20.5968, eeg_cz=-3.95115, eeg_o2=14.5076, ecg=-4520.0, r=817.705994, gsr=388.829987, event=u'A'),
 Row(crew=1, experiment=u'CA', time=0.015625, seat=1, eeg_fp1=-2.42842, eeg_f7=28.430901, eeg_f8=-9.32351, eeg_t4=-3.75723, eeg_t6=15.9693, eeg_t5=30.4436, eeg_t3=21.0103, eeg_fp2=-6.47472, eeg_o1=26.641399, eeg_p3=24.1388, eeg_pz=5.46991, eeg_f3=20.510599, eeg_fz=2.83354, eeg_f4=7.10177, eeg_c4=31.170799, eeg_p4=19.3997, eeg_poz=19.689501, eeg_c3=21.3547, eeg_cz=1.33212, eeg_o2=17.7502, ecg=-4520.0, r=817.705994, gsr=388.829987, event=u'A'),
 Row(crew=1, experiment=u'CA', time=0.01953125, seat=1, eeg_fp1=10.6715, eeg_f7=30.4202,

In [0]:
readings.printSchema()

root
 |-- crew: integer (nullable = true)
 |-- experiment: string (nullable = true)
 |-- time: double (nullable = true)
 |-- seat: integer (nullable = true)
 |-- eeg_fp1: double (nullable = true)
 |-- eeg_f7: double (nullable = true)
 |-- eeg_f8: double (nullable = true)
 |-- eeg_t4: double (nullable = true)
 |-- eeg_t6: double (nullable = true)
 |-- eeg_t5: double (nullable = true)
 |-- eeg_t3: double (nullable = true)
 |-- eeg_fp2: double (nullable = true)
 |-- eeg_o1: double (nullable = true)
 |-- eeg_p3: double (nullable = true)
 |-- eeg_pz: double (nullable = true)
 |-- eeg_f3: double (nullable = true)
 |-- eeg_fz: double (nullable = true)
 |-- eeg_f4: double (nullable = true)
 |-- eeg_c4: double (nullable = true)
 |-- eeg_p4: double (nullable = true)
 |-- eeg_poz: double (nullable = true)
 |-- eeg_c3: double (nullable = true)
 |-- eeg_cz: double (nullable = true)
 |-- eeg_o2: double (nullable = true)
 |-- ecg: double (nullable = true)
 |-- r: double (nullable = true)
 |-- gsr: do

In [0]:
readings.groupBy("experiment","event").agg({"crew":"count"}).alias("count").show()

+----------+-----+-----------+
|experiment|event|count(crew)|
+----------+-----+-----------+
|        DA|    D|     235329|
|        CA|    A|       5690|
|        DA|    A|    1423064|
|        SS|    A|    1420055|
|        CA|    C|    1652686|
|        SS|    B|     130597|
+----------+-----+-----------+



In [0]:
readings.columns[4:-3]

['eeg_fp1',
 'eeg_f7',
 'eeg_f8',
 'eeg_t4',
 'eeg_t6',
 'eeg_t5',
 'eeg_t3',
 'eeg_fp2',
 'eeg_o1',
 'eeg_p3',
 'eeg_pz',
 'eeg_f3',
 'eeg_fz',
 'eeg_f4',
 'eeg_c4',
 'eeg_p4',
 'eeg_poz',
 'eeg_c3',
 'eeg_cz',
 'eeg_o2',
 'ecg']

In [0]:
#readings.selectExpr('percentile(eeg_fp1, 0.95)').show()
#pctl=readings.percentile(["eeg_fp1", "eeg_f7", "eeg_f8"], [0.03, 0.97])

#http://blog.madhukaraphatak.com/statistical-data-exploration-spark-part-3/

pctls = {}
for col in readings.columns[4:-3]:
    print (col+"...")
    pctls[col] = readings.approxQuantile(col,[0.01,0.99],0.20)

eeg_fp1...
eeg_f7...
eeg_f8...
eeg_t4...
eeg_t6...
eeg_t5...
eeg_t3...
eeg_fp2...
eeg_o1...
eeg_p3...
eeg_pz...
eeg_f3...
eeg_fz...
eeg_f4...
eeg_c4...
eeg_p4...
eeg_poz...
eeg_c3...
eeg_cz...
eeg_o2...
ecg...


In [0]:
readings_copy=readings
print (readings.count())
for col in readings.columns[4:6]:
    readings_tmp=readings_copy.filter((readings[col]>pctls[col][0]) & (readings[col]<pctls[col][1]))
    readings_copy=readings_tmp
    print (col, " ", readings_copy.count())